# Проект. Поиск прибыльных нефтяных месторождений. 

## Глава 0. Входные данные
### Описание проекта

Для добывающей компании «ГлавРосГосНефть» нужно решить, где бурить новую скважину.

Предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализировать возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

#### Путь к файлам:
- /datasets/geo_data_0.csv
- /datasets/geo_data_1.csv
- /datasets/geo_data_2.csv

#### Признаки:
`id` — уникальный идентификатор месторождения;  
`f0`, `f1`, `f2` — три признака точек (неважно, что они означают, но сами признаки значимы); 

#### Целевой признак:
`product` — объём запасов в месторождении (тыс. баррелей).

#### Условия задачи:
 - Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
 - При разведке региона проводится исследование 500 точек.
 - Бюджет на разработку месторождений — 10 млрд рублей, стоимость бурения одной скважины — 50 млн рублей.
 - Один баррель сырья приносит 450 рублей прибыли.
 - Не рассматривать регионы, в которых риск убытков выше 2.5%. Из оставшихся выбирается регион с наибольшей средней прибылью.

## Глава 1. Загрузка и подготовка данных

Вызовем все необходимые библиотеки для реализации проекта:

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from scipy import stats as st
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

Напишем функции first_view для ознакомления с каждой из таблиц и вызовем ее для каждой

In [2]:
def first_view(df):
    print(df.info()) 
    return df.head(10)

Прочитаем файлы, создадим ДатаФреймы, через функцию выведем информацию и первые 10 строк таблиц

In [3]:
df_0 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_0.csv')
first_view(df_0)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
5,wX4Hy,0.969570,0.489775,-0.735383,64.741541
6,tL6pL,0.645075,0.530656,1.780266,49.055285
7,BYPU6,-0.400648,0.808337,-5.624670,72.943292
8,j9Oui,0.643105,-0.551583,2.372141,113.356160
9,OLuZU,2.173381,0.563698,9.441852,127.910945


In [4]:
df_1 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_1.csv')
first_view(df_1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
5,HHckp,-3.327590,-2.205276,3.003647,84.038886
6,h5Ujo,-11.142655,-10.133399,4.002382,110.992147
7,muH9x,4.234715,-0.001354,2.004588,53.906522
8,YiRkx,13.355129,-0.332068,4.998647,134.766305
9,jG6Gi,1.069227,-11.025667,4.997844,137.945408


In [5]:
df_2 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_2.csv')
first_view(df_2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
5,LzZXx,-0.758092,0.710691,2.585887,90.222465
6,WBHRv,-0.574891,0.317727,1.773745,45.641478
7,XO8fn,-1.906649,-2.458350,-0.177097,72.480640
8,ybmQ5,1.776292,-0.279356,3.004156,106.616832
9,OilcN,-1.214452,-0.439314,5.922514,52.954532


### Выводы по главе 1:

>В таблицах отсутствуют пропуски. Все данные, кроме `id` имеют численное значение. Столбец `id` не несет информации для обучения модели. 

## Глава 2. Обучение и проверка модели

 Удалим столбец `id` из обучающей выборки.

In [6]:
df_0 = df_0.drop(['id'], axis=1)
df_1 = df_1.drop(['id'], axis=1)
df_2 = df_2.drop(['id'], axis=1)

Разделим датафреймы на 2 части: обучающую `df_train` и валидационную `df_valid` выбороки в пропорциях 3:1, используя функцию `train_test_split` из библиетеки `sklearn.model_selection`

In [7]:
train_0, valid_0 = train_test_split(df_0, test_size=0.25, random_state=12345)
train_1, valid_1 = train_test_split(df_1, test_size=0.25, random_state=12345)
train_2, valid_2 = train_test_split(df_2, test_size=0.25, random_state=12345)

Выведем размеры обучающей и валидационной выборок на экран и оценим правильность деления на части

In [8]:
print(train_0.shape, valid_0.shape)
print(train_1.shape, valid_1.shape)
print(train_2.shape, valid_2.shape)

(75000, 4) (25000, 4)
(75000, 4) (25000, 4)
(75000, 4) (25000, 4)


Разделим каждую выборку на `features` -  признаки и `target` — целевой признак. Напишем для этого функцию `features_target_split()`:

In [9]:
def features_target_split(data):
    features = data.drop(['product'], axis=1)
    target = data['product']
    return features, target

train_features_0, train_target_0 = features_target_split(train_0)
train_features_1, train_target_1 = features_target_split(train_1)
train_features_2, train_target_2 = features_target_split(train_2)

valid_features_0, valid_target_0 = features_target_split(valid_0)
valid_features_1, valid_target_1 = features_target_split(valid_1)
valid_features_2, valid_target_2 = features_target_split(valid_2)

Для того, чтобы отмаштабировать признаки, воспользуемся одним из методов масштабирования — стандартизации данных.  
В sklearn есть отдельная структура для стандартизации данных — **StandardScaler**  
Преобразуем данные функцией `transform()`. Напишем для этого функцию `data_to_StandardScaler(data)`

In [10]:
def data_to_StandardScaler1(data, train_features):
    scaler1 = StandardScaler()
    scaler1.fit(train_features)
    data = scaler1.transform(data)
    return data

valid_features_0 = data_to_StandardScaler1(valid_features_0, train_features_0)
valid_features_1 = data_to_StandardScaler1(valid_features_1, train_features_1)
valid_features_2 = data_to_StandardScaler1(valid_features_2, train_features_2)

def data_to_StandardScaler(data):
    scaler = StandardScaler()
    scaler.fit(data)
    data = scaler.transform(data)
    return data

train_features_0 = data_to_StandardScaler(train_features_0)
train_features_1 = data_to_StandardScaler(train_features_1)
train_features_2 = data_to_StandardScaler(train_features_2)

Данная задача относиться к задачам регрессии. Обучим модель Линейная регрессия для каждого региона:  
 - **Линейная регрессия** LinearRegression из библиотеки *sklearn.linear_model*   

Напишем функцию `model_rmse`, принимающая данные обучающей и валидационной выборок, выводит RMSE модели и возвращает массив предсказаний модели по региону
 


In [11]:
def model_rmse(train_features, train_target, valid_features, valid_target):
    model = LinearRegression().fit(train_features, train_target)
    predicted = model.predict(valid_features)
    mse_0 = mean_squared_error(valid_target, predicted)
    rmse_0 = mse_0 ** 0.5
    print('RMSE модели {:.2f}'.format(rmse_0))
    return predicted

### Регион 0

In [12]:
print('Регион 0')
print('Cредний запас сырья в регионе {:.2f} тыс. баррелей'.format(df_0['product'].mean()))
predicted_valid_0  = model_rmse(train_features_0, train_target_0, valid_features_0, valid_target_0)

Регион 0
Cредний запас сырья в регионе 92.50 тыс. баррелей
RMSE модели 37.58


### Регион 1

In [13]:
print('Регион 1')
print('Cредний запас сырья в регионе {:.2f} тыс. баррелей'.format(df_1['product'].mean()))
predicted_valid_1  = model_rmse(train_features_1, train_target_1, valid_features_1, valid_target_1)

Регион 1
Cредний запас сырья в регионе 68.83 тыс. баррелей
RMSE модели 0.89


### Регион 2

In [14]:
print('Регион 2')
print('Cредний запас сырья в регионе {:.2f} тыс. баррелей'.format(df_2['product'].mean()))
predicted_valid_2  = model_rmse(train_features_2, train_target_2, valid_features_2, valid_target_2)

Регион 2
Cредний запас сырья в регионе 95.00 тыс. баррелей
RMSE модели 40.03


### Выводы по главе 2:

> Обучение модели Линейная регрессия в 3 регионах показала, что в регионе 1 лучшая RMSE, модель предсказывает наличие запасов нефти с точность до ~~0.9~~ <b>0.89</b> тыс.баррелей, в то время как в регионе 0 и 2 достигает около 40 тыс.баррелей.  
Однако, в регионе 1 средний запас сырья - почти 69 тыс.баррелей, а в 0 и 2 - 92 и 95 тыс.баррелей соответственно, что в 1.5 раза примерно выше региона 1.  
Модель хорошо обучилась и лучше предсказывает в регионе 1.

## Глава 3. Подготовка к расчёту прибыли

Для дальнейших расчетов нужно привести ключевые значения к пригодным для работы с ними.

In [15]:
# Сбросим индексы целевого признака в валидационной выборке для каждого региона:
valid_target_0.reset_index(drop=True, inplace=True)
valid_target_1.reset_index(drop=True, inplace=True)
valid_target_2.reset_index(drop=True, inplace=True)

# Переведем предсказания в валидационной выборке в pd.Series:
predicted_valid_0 = pd.Series(predicted_valid_0)
predicted_valid_1 = pd.Series(predicted_valid_1)
predicted_valid_2 = pd.Series(predicted_valid_2)

Чтобы рассчитать минимальный средний объём сырья в месторождениях региона, достаточный для его разработки, необходимо провести рассчеты:  
- Операционная прибыль = Валовая прибыль -  Операционные расходы(себестоимость добычи, зарплаты, аренда и пр. связанное с деятельность по добыче)  
- Один баррель сырья приносит ~~4500~~ <b>450</b> рублей прибыли. <b>Доход с каждой единицы продукта составляет 450 тыс. рублей(450000), поскольку объём указан в тысячах баррелей</b>
- Бюджет на разработку месторождений — 10 млрд рублей, стоимость бурения одной скважины — 50 млн рублей.
- Минимальный средний объём сырья в месторождениях региона, достаточный для его разработки - объем баррелей в скважине, при котором операционная прибыль = 0

In [23]:
#далее все цифры приведены к млрд. что бы не путаться
price_per_unit = 0.00045e9
budget_for_deposit = 10e9
price_per_borehole = 0.05e9

In [17]:
#x = 50000000 / 4500 / 1000
x = price_per_borehole / price_per_unit
print('Минимальный средний объём сырья {:.3f} тыс.баррелей'.format(x))

Минимальный средний объём сырья 111.111 тыс.баррелей


Для того чтобы покрыть операционные расходы по добычи в каждой скважине региона должно быть в среднем ~~11.11~~ <b>111.111</b> тыс.баррелей нефти.

Напишем функцию для расчёта прибыли по набору отобранных месторождений и предсказаний модели `distribution_revenue`, принимающая на вход предсказания, истинные значения целевого признака и количество скважин и считающая общую прибыль в млрд.руб в регионе:

In [18]:
def revenue(target, probabilities, count):
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    #return 4500 * 1000 * selected.sum() / 1e9
    return ((price_per_unit * selected.sum() - budget_for_deposit) / 1e9)

### Выводы по главе 3:

> Данные подготовлены для Bootstrap и статистического анализа. 

## Глава 4. Расчёт прибыли и рисков 

Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.  
Найдем среднюю прибыль, 95%-й доверительный интервал и риск убытков.

Напишем функцию `distribution_revenue` с техникой Bootstrap с 1000 повторениями для 500 возможных скважин, 200 из которых будут отобраны для бурения.  
Функция считает среднюю прибыль, 95%-й доверительный интервал и риск убытков по целевому признаку и предсказаниям.

In [19]:
state = np.random.RandomState(12345)
def distribution_revenue(target, probabilities):
    values = []
    for i in range(1000):
        target_subsample = target.sample(n=500, replace=True, random_state=state)
        probs_subsample = probabilities[target_subsample.index]
        values.append(revenue(target_subsample, probs_subsample, 200))

    values = pd.Series(values)
    mean = values.mean()
    print('Средняя прибыль {} млрд.руб.'.format(mean))
    
    lower = np.quantile(values, 0.025)
    upper = np.quantile(values, 0.975)
    print('95%-й доверительный интервал {} - {} млрд.руб.'.format(lower, upper))
    
    loss_risk = st.percentileofscore(values, 0)
    print('Риск убытков {} %'.format(loss_risk))
    return 

### Регион 0

In [20]:
distribution_revenue(valid_target_0, predicted_valid_0)

Средняя прибыль 0.42593852691059236 млрд.руб.
95%-й доверительный интервал -0.10209009483793655 - 0.9479763533583688 млрд.руб.
Риск убытков 6.0 %


### Регион 1

In [21]:
distribution_revenue(valid_target_1, predicted_valid_1)

Средняя прибыль 0.5182594936973248 млрд.руб.
95%-й доверительный интервал 0.1281232314330863 - 0.9536129820669086 млрд.руб.
Риск убытков 0.3 %


### Регион 2

In [22]:
distribution_revenue(valid_target_2, predicted_valid_2)

Средняя прибыль 0.4201940053440501 млрд.руб.
95%-й доверительный интервал -0.11585260916001143 - 0.989629939844574 млрд.руб.
Риск убытков 6.2 %


### Выводы по главе 4:

> Полученные данные показывают:
- при Bootstrap с 1000 выборок, распределение средних прибылей стремиться к нормальному со средним в районе ~~104~~ <b>0.423</b> млрд. для регионов 0 и 2, и в районе ~~105~~ <b>0.518</b> млрд. для региона 1. Что предполагает такое же среднее в генеральной совокупности для данной модели и региона.
- Нижний предел 95% доверительный интервал только для региона 1 выше ~~100~~ <b>0</b> млрд.рублей~~, операционная прибыль меньше 10 млрд. ни в одном из  регионов.~~
- Риски получения убытков ~~отстутствуют во всех 3 регионах~~ <b>ниже 2.5% только у региона 1</b>.
- Наиболее предпочтительным регионом для бурения скважин  по доверительному интервалу,среднему и риску убытков - Регион 1.

### Общий вывод:

- Обучение модели Линейная регрессия в 3 регионах показала, что в регионе 1 лучшая RMSE, модель предсказывает наличие запасов нефти с точность до 0.89 тыс.баррелей, в то время как в регионе 0 и 2 достигается разброс около 40 тыс.баррелей.  
- <b>В общем все регионы не подходят по среднему количеству баррелей в скважинах, т.к. все средние ниже 111.111 тыс. барелей</b>
- <b>Однако, на случайной выборке из 1000 повторений модель предсказывает среднюю прибыль и 95%-й доверительный интервал операционного дохода в Регионе 1.</b>
- <b>Очевидно что в топ 200 скважин Региона 1 по среднему количеству баррелей в скважинах выше чем общее среднее всех скважин Региона 1.</b>
- <b>Очевидны преимущества машиного обучения и конкретно нашей модели, позволившие определить перспективный регион, который на первый взгляд обладает "плохими показателями" (низким общим средним кол-вом баррелей всего региона), а так же определить существенно более высокие риски (в 20 раз выше) разработки регионов с более "выйграшными показателями" (общее среднее кол-во баррелей в полтора раза выше).</b>
- Следовательно, чтобы получить максимальную прибыль при минимальных рисках необходимо использовать Регион 1 для добычи сырья, т.к. статистически это лучший регион по заданным входным данным.